<a href="https://colab.research.google.com/github/pramod-nitk/Predict-loan-senctioned-amount-/blob/master/intent_identification_for_a_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import falcon
import pickle
import numpy as np
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from autocorrect import spell
import nltk
import spacy
import en_core_web_sm
from gensim import corpora, models
import gensim
import stop_words
nlp = en_core_web_sm.load()

C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
t = pd.read_csv('bank4.csv',low_memory=False,encoding = 'latin1')
pp = pd.read_csv('preprocess.csv',low_memory=False)

In [ ]:
folder =""
folder_mod =""

In [ ]:
folder_mod

''

In [ ]:
t.head()

,Line,Final intent
0,Hi,greetings
1,Hi. My name is,greetings
2,I am facing issues with my credit card,others
3,please help waive my annual membership,Annual Fee Reversal
4,please reverse my annual charges,Annual Fee Reversal


In [ ]:
t.columns = ["line","class1"]

In [ ]:
t['numw'] = t.line.str.count(" ")+1
#t5=t_sup1.ix[t_sup1.numw>=3,:]
t['class1'].value_counts()

greetings               1525
Foreign Travel           700
Card Delivery            600
Report Fraud             600
branch_atm_locator       550
Track application        450
Credit Limit related     400
Pin related              400
Card Cancellation        350
Card Activation          350
Blocked Card             350
Payment related          300
others                   227
Annual Fee Reversal      150
Statement related        150
Name: class1, dtype: int64

In [ ]:
cols = ["line","class1"]
newdf = pd.DataFrame(columns = cols )
newcnts = pd.DataFrame(t["class1"].value_counts()).reset_index()
newcnts.columns = ["labels","counts"]

In [ ]:
newcnts

,labels,counts
0,greetings,1525
1,Foreign Travel,700
2,Card Delivery,600
3,Report Fraud,600
4,branch_atm_locator,550
5,Track application,450
6,Credit Limit related,400
7,Pin related,400
8,Card Cancellation,350
9,Card Activation,350


In [ ]:
t1 = pd.merge(t,newcnts,left_on="class1",right_on = "labels",how="inner")

In [ ]:
t2 = t1.ix[t1.counts>=5,:]

C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [ ]:
min1=400
for index,row in newcnts.iterrows():
    lbl = row["labels"]
    if row["counts"] >=min1:
        sset = t2.ix[t2.class1==lbl,cols]
        newdf = pd.concat([newdf,sset.sample(n=min1)])
        
    else:
        num_times = min1/row["counts"]
        
        #row["counts"]/min1
        for j in range(0,num_times):
            sset = t2.ix[t2.class1==lbl,cols]
            newdf = pd.concat([newdf,sset])

C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [ ]:
newdf.ix[:,"class1"]=newdf["class1"].str.lstrip()
newdf.ix[:,"class1"]=newdf["class1"].str.rstrip()
newdf.ix[:,"class1"]=newdf["class1"].str.lower()

#print t5["Tags"].str.lower()
newdf['class1'].value_counts()

C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


report fraud            400
card delivery           400
foreign travel          400
branch_atm_locator      400
pin related             400
track application       400
greetings               400
credit limit related    400
blocked card            350
card cancellation       350
card activation         350
annual fee reversal     300
statement related       300
payment related         300
others                  227
Name: class1, dtype: int64

In [ ]:
newdf.head()

,line,class1
209,hey phone number is 6200122990,greetings
1291,goodmorning cell number 9334973609,greetings
965,good afternoon phone number is 1426749385,greetings
1213,hello madam my number is 919134590508,greetings
343,hello sir cell number 8763674117,greetings


In [ ]:
def add_len(newstr):
    ch_len = len(newstr)
    w_len = newstr.count(" ") +1 
    
    if(ch_len<=5):
        cl_bkt = "buck_0-5"
    
    elif(ch_len<=10):
        cl_bkt = "buck_5-10"
    
    
    elif(ch_len<=20):
        cl_bkt = "buck_10-20"
    
    elif(ch_len<=30):
        cl_bkt = "buck_20-30"
    else:
        cl_bkt = "long"
    
    if(w_len<=1):
        w_bkt = "bkt_single"
    elif(w_len<=3):
        w_bkt = "bkt_short"
    elif(w_len<=7):
        w_bkt = "bkt_medium"
    elif(w_len<=10):
        w_bkt = "bkt_long"
    else:
        w_bkt = "bkt_very_long"
    
    return cl_bkt,w_bkt

In [ ]:
def ext_nn(l1):
    str1 = ""
    doc5 = nlp(unicode(l1))
    for np in doc5.noun_chunks:
        str1 = str1 + " " + str(np)
        #print np
    return str1
        

In [ ]:
ext_nn("please activate my card")

' my card'

In [ ]:
str1 = "i am good"
doc1 = nlp(unicode(str1))
for i in doc1:
    print np.sum(i.vector),i.vector_norm
print "doc",doc1.vector_norm,doc1.vector.shape

0.7476568 24.211834
10.486708 27.161102
-2.7070062 24.949709
doc 14.9734354153 (384L,)


In [ ]:
stp = stop_words.get_stop_words('en')

In [ ]:

for i in str1.split():
    if(i not in set(stp)):
        print i

good


In [ ]:
import re
def preprocess(l1):
    nns = ext_nn(l1)
    newstr=[]
    
    l1=l1.replace("inr","rs")
    l1=l1.replace("\r"," ")
    l1=l1.replace("\n"," ")
    l1=re.sub('[\s]+,'," ",l1)
    l1=re.sub('http[0-9A-Za-z:\/\/\.\?\=]*',' url_pp ',l1)
    l1=re.sub('[0-9]+\/[0-9]+\/[0-9]+',' date_pp ',l1)
    l1=re.sub('91[7-9][0-9]{9}', ' mobile_pp ', l1)
    l1=re.sub('[7-9][0-9]{9}', ' mobile_pp ', l1)
    
    #patterns for landline numbers
    
    
    l1=re.sub('[0-9]+%', ' digits_percent_pp ', l1)
    l1=re.sub('[0-9]+th', ' digits_th_pp ', l1)
    l1= re.sub('rs[., ]*[0-9]+[,.]?[0-9]+[,.]?[0-9]+[,.]?[0-9]+[,.]?',' money_digits_pp ',l1)
    l1= re.sub('rs[., ]*[0-9]+',' money_digits_small_pp ',l1)
    l1= re.sub('rs[., ]*[0-9]+',' money_digits_small_pp ',l1)
    
    #l1= re.sub('inr[., ]*[0-9]+[,.]?[0-9]+[,.]?[0-9]+[,.]?[0-9]+[,.]?','money_digits',l1)
    l1 = re.sub('[0-9]+[x]+[0-9]*',' cardnum_pp ',l1)
    l1 = re.sub('[x]+[0-9]+',' cardnum_pp ',l1)
    l1 = re.sub('[0-9]{4,7}',' simp_digit_otp ',l1)
    l1 = re.sub('[0-9]+',' simp_digit_pp ',l1)
    l1=l1.replace("a/c"," ac_pp ")
    l1= re.sub('[^a-z _]',' ',l1)
    l1=re.sub('[\s]+,'," ",l1)
    l1=re.sub('[^A-Za-z]+', ' ', l1)
    for i in l1.split():
        i = spell(i)
        fg=0
        for index,rows in pp.iterrows():
            w=rows["word"]
            c=rows["class"]
            w=w.lower().lstrip().rstrip()
            if(i==w):
                newstr.append(c)
                fg=1
                break;
        if(fg==0):
            newstr.append(i)
    newstr=' '.join(newstr)
    
    #l1,w1 = add_len(newstr)
    
    nns = re.sub('[^a-zA-Z0-9\s]+','',nns)
    
    #print nns
    
    #newstr = newstr + l1 + " " + w1 + " " + unicode(nns)
    newstr = newstr +" " + unicode(nns)
    
    newstr1 = ""
    
    for n in newstr.split():
        if(n not in set(stp)):
            newstr1 = newstr1 + " " + n
    
    
    
    return newstr1

In [ ]:
preprocess('please activate my card')

u' please activate cntry card card'

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
stemmer2 = SnowballStemmer("english", ignore_stopwords=True)
ll=[]
doc_ll = []
newdf1=newdf.ix[:,"line"]
for l1 in newdf1:
    l1 = l1.lower()
    #print l1
    l1= preprocess(l1)
    
    doc1 = nlp(unicode(l1))
    
    d2v = doc1.vector
    
    doc_ll.append(d2v)
    
    #l1 =stemmer.stem(l1)
    ll.append(l1)


C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel\__main__.py:40: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [ ]:
doc_ll1 = np.array(doc_ll)

In [ ]:
doc_ll1.shape

(5277L, 384L)

In [ ]:
ll[0:5]

[u' goodmorning phone number mobile pp simp digit pp phone number',
 u' good morning simp digit top simp digit pp good morning',
 u' good morning number mobile pp simp digit pp good morning number',
 u' hi number mobile pp simp digit pp number',
 u' hello madam number mobile pp simp digit pp madam number']

In [ ]:
newdf["pp"]=ll
newdf = newdf.reset_index()

In [ ]:
newdf.head()

,index,line,class1,pp
0,714,goodmorning phone number 916383697186,greetings,goodmorning phone number mobile pp simp digit...
1,1248,good morning 1860452650,greetings,good morning simp digit top simp digit pp goo...
2,1096,good morning here is my number 916962636607,greetings,good morning number mobile pp simp digit pp g...
3,472,hi here is my number 912680305549,greetings,hi number mobile pp simp digit pp number
4,784,hello madam here is my number 913620149662,greetings,hello madam number mobile pp simp digit pp ma...


In [ ]:
tgt3=newdf['class1']

In [ ]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(tgt3, 1, test_size=0.2,random_state=42)
for train_index,test_index in sss:
    train_index = train_index
    test_index = test_index

C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
ll = list(newdf.ix[train_index,"pp"])
ll_test = list(newdf.ix[test_index,"pp"])


C:\Users\nitki\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [ ]:
doc_ll_train = doc_ll1[train_index]
print doc_ll_train.shape
doc_ll_test = doc_ll1[test_index]
print doc_ll_test.shape

(4221L, 384L)
(1056L, 384L)


In [ ]:
ll1 = []
for i in ll:
    ll1.append(i.split())
    

In [ ]:
ll1_test = []
for i in ll_test:
    ll1_test.append(i.split())
    

In [ ]:
dictionary = corpora.Dictionary(ll1)
dictionary.filter_extremes(no_below=20, no_above=0.5)

corpus = [dictionary.doc2bow(i) for i in ll1]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary, passes=5,chunksize=100)


corpus1 = ldamodel.get_document_topics(corpus, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)


corpus_test = [dictionary.doc2bow(i) for i in ll1_test]
corpus1_test = ldamodel.get_document_topics(corpus_test, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)

In [ ]:
num_topics = ldamodel.num_topics

In [ ]:
arr1 = np.zeros([len(ll1),num_topics-1])
arr1_test = np.zeros([len(ll1_test),num_topics-1])

In [ ]:
arr1.shape

(4221L, 29L)

ind=-1
for i in corpus1:
    ind=ind+1
    for j in i:
        for k in range(0,29):
            if(j[0]==k):
                #print "yes"
                arr1[ind][k]=j[1]
                break;
ind=-1
for i in corpus1_test:
    ind=ind+1
    for j in i:
        for k in range(0,29):
            if(j[0]==k):
                #print "yes"
                arr1_test[ind][k]=j[1]
                break;
               
        

In [ ]:
arr1[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
arr1_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
ll[0:50]

[u' simp digit pp weeks want know order status application 12 weeks order status application',
 u' paying URL website erred url website',
 u' ordered card last week card',
 u' please help complain wrong trans wrong trans',
 u' forgot atm pin atm pin',
 u' close atm machine located close atm machine',
 u' moving cancel account account',
 u' want complain going another country another country',
 u' trans failed trans',
 u' making payment simp digit top failed payment',
 u' waive annual membership annual membership',
 u' please creditcard personal index number creditcard personal index number',
 u' day received card can make work card',
 u' travelling cntry simp digit pp months can use card card',
 u' will receive credit card credit card',
 u' transaction go tranaction',
 u' can mail last year bill last year bill',
 u' please help credit card pin credit card pin',
 u' good evening number mobile pp good evening number',
 u' reach city bank atm citi bank atm',
 u' please send last month sta

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df = 5,ngram_range=(1, 4),stop_words ='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(ll)

tf1= tfidf_matrix.todense()

tf1_test = tfidf_vectorizer.transform(ll_test)
tf1_test=tf1_test.todense()
len(tf1),tf1.shape,tf1_test.shape

(4221, (4221L, 2333L), (1056L, 2333L))

In [ ]:
tfidf_vectorizer.vocabulary_

{u'digit available balance': 683,
 u'past simp digit pp': 1552,
 u'rs wrong': 1844,
 u'mam mobile': 1320,
 u'machine proximate': 1269,
 u'status app': 2007,
 u'stopped charge': 2066,
 u'digit': 681,
 u'closest branch closest': 521,
 u'applied order status': 123,
 u'proximate citi bank': 1728,
 u'creditcard europe creditcard': 623,
 u'pp number number': 1682,
 u'simp digit card': 1944,
 u'card received': 364,
 u'card status': 372,
 u'88888 rupees': 32,
 u'weeks order status': 2293,
 u'proximity atm machine': 1733,
 u'let know set': 1219,
 u'frustrating cancel account account': 887,
 u'near cash': 1420,
 u'set travel alert travel': 1901,
 u'creditcard personal index': 639,
 u'mobile pp simp': 1353,
 u'citibank cash machine citibank': 454,
 u'increase credit credit': 1159,
 u'url website': 2186,
 u'status app order': 2008,
 u'good night number': 944,
 u'send month statement': 1876,
 u'locked': 1251,
 u'creditcard help creditcard': 629,
 u'digit pp weeks use': 749,
 u'want': 2211,
 u'pp da

In [ ]:
tfidf_vectorizer_char = TfidfVectorizer(min_df = 10,ngram_range=(2, 6),analyzer ='char')
tfidf_matrix_char = tfidf_vectorizer_char.fit_transform(ll)
tf1_char = tfidf_matrix_char.todense()

tf1_test_char = tfidf_vectorizer_char.transform(ll_test)
tf1_test_char=tf1_test_char.todense()

len(tf1_char),tf1_char.shape,tf1_test_char.shape

(4221, (4221L, 7371L), (1056L, 7371L))

In [ ]:
doc_ll_test.shape,tf1.shape

((1056L, 384L), (4221L, 2333L))

In [ ]:

tf1_all = np.concatenate([tf1,tf1_char,arr1,doc_ll_train],axis=1)
#tf1_all = np.column_stack ([tf1,tf1_char,arr1,doc_ll_train])
tf1_test_all = np.concatenate([tf1_test,tf1_test_char,arr1_test,doc_ll_test],axis=1)
#tf1_test_all = np.column_stack([tf1_test,tf1_test_char,arr1_test,doc_ll_test])
tf1_all.shape,tf1_test_all.shape

((4221L, 10117L), (1056L, 10117L))

In [ ]:
import numpy as np
a_train_b,a_test_b= tf1_all,tf1_test_all
b_train_b= tgt3[train_index]
b_test_b = tgt3[test_index]

print isinstance(b_test_b,np.ndarray)

False


In [ ]:
#import pickle
#pickle.dump(tfidf_vectorizer, open(flop + "/" + "tfidf_vectorizer", "wb"))

from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=60)

#selector.fit(a_train_b, b_train_b)
a_train_b = selector.fit_transform(a_train_b, b_train_b)
a_test_b = selector.transform(a_test_b)

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=5)
#selector.fit(a_train_b, b_train_b)
a_train_b_np = selector.fit_transform(a_train_b, b_train_b)
a_train_b=pd.DataFrame(a_train_b_np)
a_test_b = selector.transform(a_test_b)

In [ ]:
#pickle.dump(selector, open(flop + "/" + "selector", "wb"))
a_train_b.shape
type(a_train_b)

numpy.ndarray

In [ ]:
from sklearn.linear_model import LogisticRegression
#sample_weight = {"driving_re":0.5,"stil_re":2,"other":2,"walking_re":0.75}
#sample_weight = {"driving_re":0.5,"other":1}
#class_weight=sample_weight
clf_log = LogisticRegression()

clf_log.fit(a_train_b,b_train_b)  

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
pred=clf_log.predict(a_test_b)
from sklearn.metrics import accuracy_score
print accuracy_score(b_test_b, pred)

0.9450757575757576


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb= GradientBoostingClassifier(n_estimators=300)
gb.fit(a_train_b,b_train_b)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [ ]:
pred=gb.predict(a_test_b)
from sklearn.metrics import accuracy_score
print accuracy_score(b_test_b, pred)

0.9876893939393939


In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
scores_lr = cross_val_score(clf_log, a_train_b,b_train_b, cv=3, scoring='accuracy')
scores_gb = cross_val_score(gb, a_train_b,b_train_b, cv=3, scoring='accuracy')

In [ ]:
print scores_lr,scores_gb

[0.93413598 0.94677076 0.93571429] [0.98087819 0.98296664 0.97142857]


In [ ]:
#pickle.dump(clf_log, open(flop + "/" + "clf_log", "wb"))

In [ ]:
from sklearn.externals import joblib

joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.joblib",compress=1)
joblib.dump(tfidf_vectorizer_char, "tfidf_vectorizer_char.joblib",compress=1)
joblib.dump(dictionary, "dictionary.joblib",compress=1)
joblib.dump(ldamodel,"ldamodel.joblib",compress=1)
joblib.dump(selector,"selector.joblib",compress=1)
joblib.dump(clf_log, "clf_log.joblib",compress=1)
joblib.dump(gb,"gb.joblib",compress=1)


['gb.joblib']

In [ ]:
pickle.dump(tfidf_vectorizer, open(folder_mod + "/" + "tfidf_vectorizer", "wb"))
pickle.dump(tfidf_vectorizer_char, open(folder_mod + "/" + "tfidf_vectorizer_char", "wb"))

pickle.dump(dictionary, open(folder_mod + "/" + "dictionary", "wb"))
pickle.dump(ldamodel, open(folder_mod + "/" + "ldamodel", "wb"))



pickle.dump(selector, open(folder_mod + "/" + "selector", "wb"))
pickle.dump(clf_log, open(folder_mod + "/" + "clf_log", "wb"))
pickle.dump(gb, open(folder_mod + "/" + "gb", "wb"))


IOError: [Errno 13] Permission denied: '/tfidf_vectorizer'